In [1]:
from data import initialize_loader, Flickr8k
from encoder_decoder import ResNetEncoder, Decoder, ResNetAttentionEncoder, DecoderWithAttention
import torch
from torchvision import transforms
import matplotlib.pyplot as plt
import pickle
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [3]:
args = AttrDict()
# You can play with the hyperparameters here, but to finish the assignment,
# there is no need to tune the hyperparameters here.
args_dict = {
    "learn_rate": 0.001,
    "batch_size": 32,
    "epochs": 5,
    "log_step": 15,
    "save_epoch": 1,
    "model_path": "models/",
    "load_model": False,
    "encoder_path": "models/encoder-attention-5.ckpt",
    "decoder_path": "models/decoder-attention-5.ckpt",
}
args.update(args_dict)

In [4]:
with open("vocab.pkl", 'rb') as f:
        vocab = pickle.load(f)

In [5]:
# e = ResNetEncoder(256)
# d = Decoder(len(vocab), 256, 512)
e = ResNetAttentionEncoder(256)
d = DecoderWithAttention(len(vocab), 256, 512, 512, 128)

In [6]:
# train(e, d, args)

In [7]:
transform = transforms.Compose(
            [
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
            ])

train_data = Flickr8k(csv_file="flickr8k/train.csv", root_dir="flickr8k/images", vocab=vocab, transform=transform)
train_loader = initialize_loader(train_data, batch_size=args.batch_size)
val_data = Flickr8k(csv_file="flickr8k/val.csv", root_dir="flickr8k/images", vocab=vocab, transform=transform)

In [8]:
from validation import *
e.load_state_dict(torch.load(args.encoder_path, map_location=torch.device('cpu')))
d.load_state_dict(torch.load(args.decoder_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [9]:
# for i, (imgs, captions, lengths) in enumerate(train_loader):
#     imgs = imgs.to(device)
#     captions = captions.to(device)
#     e.eval()
#     d.eval()
#     c = caption_image(e, d, imgs[0].unsqueeze(0).to(device), vocab)
#     print(c)
#     captions = bulk_caption_image(e,d,imgs, vocab)
#     l = []
#     for i in range(32 // 5):
#         l.append(captions[i*5])
#     print(l)
    # r = random.randint(0,32)
    # img = imgs[r].unsqueeze(0).to(device)
    # e.eval()
    # d.eval()
    # print(" ".join(caption_image(e, d, imgs[r].unsqueeze(0), vocab)))
    # plt.imshow(imgs[r].permute(1, 2, 0).cpu())
    # sentence = map(lambda x: train_data.vocab.itos[x], captions[r])
    # print(" ".join(sentence))
    # break

In [10]:
from validation import *
# x = p()
# print(x)
bleu_score = evaluate_bleu_batch(e, d, vocab, val_data, 
                attention=True,
                maxn_gram=1,
                batch_size=128)
# c,r = get_captions_and_references(e, d, vocab, val_data, 
#                 attention=True,
#                 batch_size=2)

In [11]:
# for i, (imgs, captions, lengths) in enumerate(train_loader):
#     imgs = imgs.to(device)
#     with torch.no_grad():
#         features = e(imgs)
#         caps = d.generate_caption(features, vocab=vocab)
#         break


In [12]:
bleu_score

0.37305066784222923